In [3]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import openpyxl
import dbfread
import glob
from pandastable import Table, TableModel # Importar clases de pandastable

### Subir reporte danilo

In [ ]:
archivo = None
while True:
    archivo=funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            
            #dbf = dbfread.DBF(ruta_archivo)
            monotributo = pd.read_excel(ruta_archivo, sheet_name=0)
            print(monotributo)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

In [ ]:
monotributo.count()

###  Importar padron total gecros monotrbutistas

In [ ]:
#cargar primero el padron de sidefi
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            padron = pd.read_excel(ruta_archivo)
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padron)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

In [ ]:
padron['CUIL'] = pd.to_numeric(padron['CUIL'], errors='coerce').fillna(0).astype('Int64')
padron['CUIL'].head()

#### Cruce de archivos! 

In [ ]:
liquidaciones_monot = pd.merge(monotributo, padron, left_on= 'cuit', right_on='CUIL', how='inner', indicator=True)
liquidaciones_monot["Razón Social"]=" "
liquidaciones_monot['periodo'] = liquidaciones_monot['periodo'].astype(str).apply(lambda x: "20" + x)
liquidaciones_monot['periodo'] = liquidaciones_monot['periodo'].astype(int)
liquidaciones_monot.columns

In [ ]:
print(liquidaciones_monot["periodo"])

In [10]:
aportes_a_importar = liquidaciones_monot[["cuit",
"Razón Social",
"periodo",
"fecha",
"CUIL",
"APELLIDO",
"NOMBRES",
"sueldo",
"aporte",
"contri",
"otrosconc"
]]

In [ ]:
path = funciones.seleccionar_directorio()
liquidaciones_monot.to_excel(path + '\cruce_desde el 15-09-23 al 30-09-23_raw.xlsx', index=False)
aportes_a_importar.to_excel(path + '\cruce_desde el 15-09-23 al 30-09-23.xls', index = False, engine='xlwt', header = False)

In [ ]:
aportes_a_importar.info()